In [1]:
# !pip install tensorflow_io
# !pip install kaggle

In [10]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
import tensorflow_io as tfio
from matplotlib import pyplot as plt
from tensorflow import keras
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.sequence import pad_sequences

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\jasmi\anaconda3\lib\site-packages\keras\preprocessing\sequence.py)

In [3]:
# Set the directory path
dir_path = 'Data/Archive_2'

# Create a list to store the audio data and labels
data = []
labels = []

# Set the maximum length of the MFCC feature matrices
max_len = 862

# Loop through the directories in the base directory
for dirpath, dirnames, filenames in os.walk(dir_path):

    # Loop through the audio files in each directory
    for filename in filenames:
        if filename.endswith('.wav'):
            # Get the full path to the audio file
            filepath = os.path.join(dirpath, filename)

            # Load the audio file
            audio, sr = librosa.load(filepath)

            # Extract the MFCC features
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)

            # Append the features and label to the data list
            data.append(mfccs)
            labels.append(os.path.basename(os.path.dirname(filepath)))

# Pad the MFCC feature matrices to the same length
data = pad_sequences(data, maxlen=max_len, padding='post', dtype='float32', truncating='post')

In [5]:
type(data)

list

In [8]:
print(data[0])

[[-154.40219   -220.9058    -429.92896   ... -433.03455   -410.77133
  -338.87708  ]
 [ 119.402985   120.652115    56.575882  ...   52.199768    74.98485
   108.66513  ]
 [  14.304213    16.447674    43.330276  ...   39.19467     45.319626
    28.103863 ]
 ...
 [   2.7608032    3.7864006    8.909393  ...    0.6091108    1.7350227
     3.3905015]
 [   1.1594774    2.7529054    7.9375834 ...    2.6871736    2.440865
     2.683006 ]
 [   2.2479491    3.152337     5.367258  ...    4.6718426    2.900693
     2.89327  ]]


In [ ]:
# Print the number of audio files and their labels
# print(f'Number of audio files: {len(data)}')
# print(f'Labels: {set(labels)}')

In [4]:
# Convert the data and labels to NumPy arrays
X = np.array(data)
le = LabelEncoder()
y = le.fit_transform(labels)
y = to_categorical(y)

C:\Users\jasmi\AppData\Local\Temp/ipykernel_7552/264575378.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(data)


ValueError: could not broadcast input array from shape (40,862) into shape (40,)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalize the data
mean = np.mean(X_train)
std = np.std(X_train)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [ ]:
# Define the CNN architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=50, verbose=1, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])